In [5]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
import scipy as sp
import pingouin as pg
sns.set_context('paper')
np.set_printoptions(suppress=True)
# pd.set_option('display.max_columns', None)

In [6]:
def simu_success(tag, params):
    # which task
    path = "simuS1_data/simuS1_design.pkl"
    if tag == "Asso-CR":
        # path = "../Data/simuS1_group1_design.pkl"
        nitems = 48*4  # 96
        test1_num = 40
        i = 1
    elif tag == "Pair-CR":
        # path = "../Data/simuS1_group2_design.pkl"
        nitems = 48*4  # 176
        test1_num = 80
        i = 2
    elif tag == "Item-CR":
        # path = "../Data/simuS1_group3_design.pkl"
        nitems = 48*4  # 136
        test1_num = 80
        i = 3

    # load stimuli
    with open(path, 'rb') as inp:
        df_study = pickle.load(inp)
        df_test = pickle.load(inp)
    df_study = df_study.query(f"group == {i}")
    df_test = df_test.query(f"group == {i}")

    # load semantic matrix
    s_mat = np.load('../../Data/wordpools/ltp_FR_similarity_matrix.npy')

    # update n_item in params to fit the study
    params.update(nitems_in_accumulator = nitems)
    # print(params)

    # run CMR
    df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat)
    df_simu['test'] = df_test['test']
    df_simu = df_simu.merge(df_test,on=['session','list','test','test_itemno1','test_itemno2'])

    # get f_in
    sessions = np.unique(df_simu.session)
    tmp_corr_fin = []
    tmp_omax_fin = []

    for sess in sessions:
        df_tmp = df_study.loc[df_study.session == sess]
        tmp1 = df_tmp.study_itemno1.to_numpy()
        tmp2 = df_tmp.study_itemno2.to_numpy()
        df_tmp2 = df_test.loc[df_test.session == sess]
        tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
        tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
        tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
        tmp = np.unique(tmp)  # sort
        nlists = len(np.unique(df_simu.list))

        for lst in range(nlists):
            tmp_corr = df_simu.query(f"session == {sess} and list == {lst}")["correct_ans"][test1_num:]
            corrid = np.searchsorted(tmp, tmp_corr)
            corr_fin = [f_in_dif[sess][lst*int(test1_num/2)+i][id] for i, id in enumerate(corrid)]
            omax_fin = [np.max(np.delete(f_in_dif[sess][lst*int(test1_num/2)+i], id)) for i, id in enumerate(corrid)]

            tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
            tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

    df_simu['corr_fin'] = tmp_corr_fin
    df_simu['omax_fin'] = tmp_omax_fin

    # optimal threshold for test1 (only when manually!!)
    # csim_two = df_simu.query("test==1").groupby("correct_ans").csim.mean()
    # opt_thresh = np.mean(csim_two)
    # df_simu['s_resp'] = df_simu.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

    return df_simu

In [7]:
# anal_perform used by pso
import sys
sys.path.append("../../Modeling/CMR_IA/fitting")
from object_funcs import anal_perform_S1 as anal_perform

In [8]:
# def anal_perform(df_simu):

#     # get correctness
#     df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans

#     # recognition performance
#     df_recog = df_simu.query("test==1")
#     hr_far = df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")
#     hr = hr_far['Yes rate'][1]
#     far = hr_far['Yes rate'][0]
#     z_hr_far = sp.stats.norm.ppf(hr_far)
#     d_prime = z_hr_far[1].item() - z_hr_far[0].item()
#     print("recognition: \n", hr_far)
#     print("d_prime: ", d_prime)

#     # cued recall performance
#     df_cr = df_simu.query("test==2")
#     p_rc = df_cr.correct.mean()
#     print("cued recall: \n", p_rc)

#     # analyze pair
#     def get_pair(df_tmp):
#         df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
#         df_tmp_pair.columns = ["test1","test2"]
#         df_tmp_pair.reset_index(inplace=True)
#         return df_tmp_pair

#     df_simu_p = df_simu.query("pair_idx >= 0")
#     df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
#     test2_rsp = pd.Categorical(df_pair.test2, categories=[0,1])
#     test1_rsp = pd.Categorical(df_pair.test1, categories=[0,1])
#     df_tab = pd.crosstab(index=test2_rsp,columns=test1_rsp, rownames=['test2'], colnames=['test1'], normalize=False, dropna=False)
#     print("contingency table: \n", df_tab)
#     # print("contingency table norm: \n", pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all'))

#     # compute" Q
#     def Yule_Q(A, B, C, D):
#         return (A * D - B * C) / (A * D + B * C)
#     q = Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)  # add 0.5
#     print("Q: ", q)

#     return p_rc, hr, far, q

### Study Params

In [9]:
params = cmr.make_default_params()
# learn while retrieving
# multiple lists, so beta_rec_post
# correct session num - done
# it seems that the size of all potential words matters, the more, the worse CR. RISK IT!

In [10]:
# # manually adjusted params
# # study params should be the same across three groups
# params.update(
#     beta_enc = 0.3,
#     gamma_fc = 0.2,
#     gamma_cf = 0.2,
#     s_fc = 0.2,
#     s_cf = 0.2,
#     phi_s = 1,
#     phi_d = 0.6,
#     d_ass = 1,
#     learn_while_retrieving = False,
#     var_enc = 1,
#     bad_enc_ratio = 1,
# )
# params.update(
#     beta_cue = 0.4,
#     beta_rec = 0.1,  # beta for retrieved item
#     beta_rec_post = 1,
#     beta_distract = 0.1,
#     # beta_rec_post = 0.1,
#     c_thresh_itm = 0.01,
#     c_thresh_ass = 0.01,
#     c_thresh = 0.01,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params

In [11]:
# pso results
import sys
sys.path.append("../../Modeling/CMR_IA/fitting")
from optimization_utils import make_boundary

_,_,what_to_fit = make_boundary(sim_name='S1')
optim_params = np.loadtxt("../../Modeling/CMR_IA/fitting/outparams_backup/S1_231102_500-200.txt")
for pname, pvalue in zip(what_to_fit, optim_params):
    params[pname] = pvalue
# params.update(beta_rec_post=1)
params.update(learn_while_retrieving=True, rec_time_limit=10000, use_new_context = True, d_ass = 0)
params

{'beta_enc': 0.441868,
 'beta_rec': 0.03125,
 'beta_cue': 0.74583,
 'beta_rec_post': 0.92021,
 'beta_distract': 0.467233,
 'phi_s': 4.801034,
 'phi_d': 2.025982,
 's_cf': 0.016863,
 's_fc': 0.218485,
 'kappa': 0.366019,
 'eta': 0.020671,
 'omega': 9.192145,
 'alpha': 0.713581,
 'c_thresh': 0.297338,
 'c_thresh_itm': 0.410577,
 'c_thresh_ass': 0.530453,
 'd_ass': 0,
 'lamb': 0.069884,
 'rec_time_limit': 10000,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'thresh_sigma': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.209973,
 'gamma_cf': 0.709904,
 'use_new_context': True}

### Association - CR (Group1)

In [12]:
tag = "Asso-CR"
df_simu_g1 = simu_success(tag, params)
df_simu_g1

CMR2 Time: 53.46412634849548


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,subject,group,corr_fin,omax_fin
0,0,0,1213,926,0.0,522.826660,0.494483,1,ROUGE,MOUTH,0,1,-1,0,1,-1.000000,-1.000000
1,0,0,830,717,0.0,2210.281982,0.398752,1,LIST,HORIZON,1,1,34,0,1,-1.000000,-1.000000
2,0,0,577,1183,0.0,778.497803,0.346577,1,FLEA,RELISH,1,1,0,0,1,-1.000000,-1.000000
3,0,0,588,439,0.0,612.767822,0.486546,1,FOLDER,DANDRUFF,1,0,25,0,1,-1.000000,-1.000000
4,0,0,687,1311,0.0,283.373505,0.525107,1,HANGER,SIRLOIN,1,1,22,0,1,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,99,4,1169,-1,1310.0,710.000000,0.618906,2,RAT,-1,331,0,226,99,1,-0.369416,2.976128
29996,99,4,1349,-1,452.0,600.000000,0.700937,2,SPHINX,-1,848,1,219,99,1,-0.384431,3.849527
29997,99,4,1307,-1,130.0,500.000000,0.723312,2,SILK,-1,72,0,194,99,1,-0.394211,4.331075
29998,99,4,984,-1,1593.0,560.000000,0.673892,2,ORGAN,-1,899,1,196,99,1,-0.377842,3.021859


In [13]:
subjects = np.unique(df_simu_g1.subject)
g1_stats = []
for subj in subjects:
    df_subj = df_simu_g1.query(f"subject == {subj}").copy()
    # df_subj = df_simu_g1.query(f"subject == {subj} and list == 0").copy()
    g1_stats.append(list(anal_perform(df_subj)))

In [14]:
print(np.array(g1_stats))
print(np.mean(np.array(g1_stats),axis=0))
print(sp.stats.sem(np.array(g1_stats),axis=0))

[[ 0.04        0.2         0.03        0.82758621]
 [ 0.02        0.17        0.1         0.66666667]
 [ 0.06        0.21        0.11        0.77707006]
 [ 0.          0.15        0.07        0.69306931]
 [ 0.03        0.19        0.06        0.94378194]
 [ 0.01        0.19        0.07        0.15827338]
 [ 0.06        0.24        0.04        0.73479948]
 [ 0.04        0.26        0.12        0.93506494]
 [ 0.06        0.17        0.09        0.83146067]
 [ 0.03        0.23        0.07        0.7114094 ]
 [ 0.02        0.27        0.09        0.46464646]
 [ 0.03        0.25        0.11        0.68171558]
 [ 0.03        0.23        0.04        0.7114094 ]
 [ 0.03        0.25        0.12        0.91833031]
 [ 0.04        0.25        0.06        0.77079796]
 [ 0.04        0.26        0.09        0.75897436]
 [ 0.05        0.26        0.04        0.81481481]
 [ 0.03        0.25        0.07        0.68171558]
 [ 0.05        0.26        0.06        0.624     ]
 [ 0.04        0.21        0.04

In [15]:
g1_stats = np.array(g1_stats)
g1_ds = sp.stats.norm.ppf(np.array(g1_stats[:,1])) - sp.stats.norm.ppf(np.array(g1_stats[:,2]))
print(np.mean(g1_ds[np.isfinite(g1_ds)]))
print(sp.stats.sem(g1_ds[np.isfinite(g1_ds)]))

0.7084970664657869
0.024593689735562106


### Pair - CR (Group2)

In [16]:
tag = "Pair-CR"
df_simu_g2 = simu_success(tag, params)
df_simu_g2

CMR2 Time: 73.0078399181366


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,subject,group,corr_fin,omax_fin
0,0,0,192,1402,0.0,1662.729004,0.436635,1,BULB,SUIT,1,1,13,0,2,-1.000000,-1.000000
1,0,0,810,1481,0.0,1100.733765,0.457259,1,LEMON,TOOTH,0,-1,-1,0,2,-1.000000,-1.000000
2,0,0,384,686,1.0,60.653149,0.602187,1,COTTAGE,HANDKERCHIEF,1,1,8,0,2,-1.000000,-1.000000
3,0,0,1317,1137,0.0,19.887587,0.163213,1,SKILLET,PUBLISHER,0,-1,-1,0,2,-1.000000,-1.000000
4,0,0,506,977,0.0,2726.044189,0.409239,1,DUNE,ONION,0,-1,-1,0,2,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,99,2,120,-1,173.0,940.000000,0.589005,2,BELT,-1,917,0,97,99,2,-0.226971,1.537186
35996,99,2,328,-1,1029.0,890.000000,0.624922,2,CLOUD,-1,869,0,113,99,2,-0.404442,1.872770
35997,99,2,456,-1,41.0,810.000000,0.598634,2,DIAGRAM,-1,1582,0,110,99,2,-0.378130,1.469504
35998,99,2,655,-1,1352.0,820.000000,0.724603,2,GRASS,-1,1520,0,114,99,2,0.265914,2.156337


In [17]:
df_simu_g2.query("session == 0 and list == 0 and test == 1")

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,subject,group,corr_fin,omax_fin
0,0,0,192,1402,0.0,1662.729004,0.436635,1,BULB,SUIT,1,1,13,0,2,-1.0,-1.0
1,0,0,810,1481,0.0,1100.733765,0.457259,1,LEMON,TOOTH,0,-1,-1,0,2,-1.0,-1.0
2,0,0,384,686,1.0,60.653149,0.602187,1,COTTAGE,HANDKERCHIEF,1,1,8,0,2,-1.0,-1.0
3,0,0,1317,1137,0.0,19.887587,0.163213,1,SKILLET,PUBLISHER,0,-1,-1,0,2,-1.0,-1.0
4,0,0,506,977,0.0,2726.044189,0.409239,1,DUNE,ONION,0,-1,-1,0,2,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0,0,1635,1509,0.0,891.841064,0.467781,1,ZEBRA,TROPHY,1,1,2,0,2,-1.0,-1.0
76,0,0,1490,1612,1.0,191.951202,0.544584,1,TOWER,WINDSHIELD,1,1,34,0,2,-1.0,-1.0
77,0,0,1153,1314,1.0,68.367058,0.596201,1,RACCOON,SKELETON,1,1,35,0,2,-1.0,-1.0
78,0,0,172,579,0.0,1848.611694,0.431336,1,BRICK,FLESH,0,-1,-1,0,2,-1.0,-1.0


In [18]:
subjects = np.unique(df_simu_g2.subject)
g2_stats = []
for subj in subjects:
    df_subj = df_simu_g2.query(f"subject == {subj}").copy()
    g2_stats.append(list(anal_perform(df_subj)))

In [19]:
print(np.array(g2_stats))
print(np.mean(np.array(g2_stats),axis=0))
print(sp.stats.sem(np.array(g2_stats),axis=0))

[[0.04166667 0.3        0.00833333 0.77031802]
 [0.03333333 0.20833333 0.00833333 0.81481481]
 [0.05       0.28333333 0.04166667 0.66593647]
 [0.03333333 0.225      0.         0.56410256]
 [0.04166667 0.25833333 0.00833333 0.81228669]
 [0.075      0.38333333 0.00833333 0.53256389]
 [0.03333333 0.325      0.01666667 0.67459138]
 [0.05       0.20833333 0.         0.88828338]
 [0.05       0.23333333 0.00833333 0.73852721]
 [0.025      0.18333333 0.         0.94499295]
 [0.04166667 0.3        0.025      0.77031802]
 [0.04166667 0.28333333 0.         0.57943925]
 [0.025      0.25       0.00833333 0.67917448]
 [0.03333333 0.175      0.         0.85100671]
 [0.05833333 0.28333333 0.         0.72609099]
 [0.04166667 0.24166667 0.00833333 0.82828283]
 [0.03333333 0.225      0.         0.7961165 ]
 [0.03333333 0.21666667 0.01666667 0.21192053]
 [0.03333333 0.225      0.01666667 0.7961165 ]
 [0.03333333 0.25       0.01666667 0.51282051]
 [0.05       0.175      0.         0.81865285]
 [0.05833333 

In [20]:
# hack hr and far
test_num = 80
g2_stats_hack = np.array(g2_stats)
hacked_hr = (g2_stats_hack[:, 1] * test_num + 0.5) / (test_num + 0.5)
hacked_far = (g2_stats_hack[:, 2] * test_num + 0.5) / (test_num + 0.5)
g2_stats_hack[:, 1] = hacked_hr
g2_stats_hack[:, 2] = hacked_far
g2_stats_hack

array([[0.04166667, 0.30434783, 0.01449275, 0.77031802],
       [0.03333333, 0.21325052, 0.01449275, 0.81481481],
       [0.05      , 0.28778468, 0.04761905, 0.66593647],
       [0.03333333, 0.22981366, 0.00621118, 0.56410256],
       [0.04166667, 0.26293996, 0.01449275, 0.81228669],
       [0.075     , 0.38716356, 0.01449275, 0.53256389],
       [0.03333333, 0.32919255, 0.02277433, 0.67459138],
       [0.05      , 0.21325052, 0.00621118, 0.88828338],
       [0.05      , 0.23809524, 0.01449275, 0.73852721],
       [0.025     , 0.1884058 , 0.00621118, 0.94499295],
       [0.04166667, 0.30434783, 0.0310559 , 0.77031802],
       [0.04166667, 0.28778468, 0.00621118, 0.57943925],
       [0.025     , 0.25465839, 0.01449275, 0.67917448],
       [0.03333333, 0.18012422, 0.00621118, 0.85100671],
       [0.05833333, 0.28778468, 0.00621118, 0.72609099],
       [0.04166667, 0.24637681, 0.01449275, 0.82828283],
       [0.03333333, 0.22981366, 0.00621118, 0.7961165 ],
       [0.03333333, 0.22153209,

In [21]:
# g2_stats = np.array(g2_stats)
g2_stats = g2_stats_hack
g2_ds = sp.stats.norm.ppf(np.array(g2_stats[:,1])) - sp.stats.norm.ppf(np.array(g2_stats[:,2]))
print(np.mean(g2_ds))
print(sp.stats.sem(g2_ds))

1.540463827343345
0.02405621630832663


### Item - CR (Group3)

In [22]:
tag = "Item-CR"
df_simu_g3 = simu_success(tag, params)
df_simu_g3

CMR2 Time: 91.09922695159912


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,subject,group,corr_fin,omax_fin
0,0,0,609,-1,1.0,588.113708,0.488600,1,FROST,NaN,1,0,14,0,3,-1.000000,-1.000000
1,0,0,101,-1,0.0,285.175995,0.296364,1,BATTERY,NaN,0,-1,-1,0,3,-1.000000,-1.000000
2,0,0,927,-1,0.0,1405.281372,0.376108,1,MOVIE,NaN,1,1,13,0,3,-1.000000,-1.000000
3,0,0,1303,-1,1.0,1174.844360,0.454001,1,SIDEWALK,NaN,1,0,31,0,3,-1.000000,-1.000000
4,0,0,945,-1,1.0,918.717102,0.466297,1,NEST,NaN,1,0,2,0,3,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,99,3,509,-1,1065.0,580.000000,0.743179,2,DUSTPAN,NaN,128,1,149,99,3,-0.367545,4.821939
47996,99,3,2,-1,475.0,490.000000,0.732745,2,ACID,NaN,970,1,163,99,3,0.060123,5.206651
47997,99,3,1463,-1,218.0,540.000000,0.674897,2,TICK,NaN,1255,0,171,99,3,0.059887,4.260575
47998,99,3,1615,-1,1276.0,520.000000,0.616554,2,WINNER,NaN,667,1,157,99,3,-0.428291,4.175273


In [23]:
subjects = np.unique(df_simu_g3.subject)
g3_stats = []
for subj in subjects:
    df_subj = df_simu_g3.query(f"subject == {subj}").copy()
    g3_stats.append(list(anal_perform(df_subj)))

In [24]:
print(np.array(g3_stats))
print(np.mean(np.array(g3_stats),axis=0))
print(sp.stats.sem(np.array(g3_stats),axis=0))

[[ 0.00625     0.5375      0.29375    -0.55855856]
 [ 0.025       0.60625     0.20625     0.21359223]
 [ 0.01875     0.50625     0.175       0.24405705]
 [ 0.025       0.6         0.225       0.2262069 ]
 [ 0.025       0.48125     0.20625     0.03821656]
 [ 0.0125      0.53125     0.24375    -0.06289308]
 [ 0.0125      0.48125     0.24375     0.69371197]
 [ 0.01875     0.4625      0.25625     0.3255814 ]
 [ 0.025       0.49375     0.175       0.42118537]
 [ 0.          0.5         0.23125     0.        ]
 [ 0.025       0.5875      0.35        0.25102319]
 [ 0.01875     0.4875      0.23125    -0.23211447]
 [ 0.0125      0.525       0.20625    -0.70020121]
 [ 0.0125      0.4875      0.25        0.68711656]
 [ 0.01875     0.5125      0.1125     -0.27943485]
 [ 0.01875     0.6125      0.25        0.03015075]
 [ 0.          0.5375      0.275      -0.07453416]
 [ 0.00625     0.5125      0.33125    -0.52307692]
 [ 0.0125      0.575       0.20625     0.58198614]
 [ 0.00625     0.5125      0.26

In [25]:
g3_stats = np.array(g3_stats)
g3_ds = sp.stats.norm.ppf(np.array(g3_stats[:,1])) - sp.stats.norm.ppf(np.array(g3_stats[:,2]))
print(np.mean(g3_ds))
print(sp.stats.sem(g3_ds))

0.7804247744599782
0.01555566314168998


In [26]:
stats = []
stats.append(list(np.mean(np.array(g1_stats),axis=0)))
stats.append(list(np.mean(np.array(g2_stats),axis=0)))
stats.append(list(np.mean(np.array(g3_stats),axis=0)))
stats = np.array(stats)
stats

array([[0.0352    , 0.2305    , 0.0781    , 0.68722551],
       [0.04325   , 0.2578882 , 0.0163147 , 0.70872911],
       [0.0168125 , 0.5228125 , 0.2373125 , 0.05862094]])

In [27]:
ground_truth = np.array([[0.42, 0.72, 0.22, 0.81], [0.30, 0.80, 0.12, 0.71], [0.19, 0.67, 0.15, 0.57]])  # p_rc, hr, far, q
err = np.mean(np.power(stats - ground_truth,2))
err

0.09285321531524188

### Performance Analysis

In [28]:
def array2df(group_stats, group_num):
    group_stats = np.array(group_stats)
    df = pd.DataFrame(group_stats, columns = ['PR','HR','FAR','Q'])
    df['d'] = sp.stats.norm.ppf(np.array(group_stats[:,1])) - sp.stats.norm.ppf(np.array(group_stats[:,2]))
    df['subject'] = df.index
    df['group'] = group_num
    return df
df_group1 = array2df(g1_stats, 1)
df_group2 = array2df(g2_stats, 2)
df_group3 = array2df(g3_stats, 3)
df_groups = pd.concat([df_group1, df_group2, df_group3])
df_groups

,PR,HR,FAR,Q,d,subject,group
0,0.04000,0.20000,0.03000,0.827586,1.039172,0,1
1,0.02000,0.17000,0.10000,0.666667,0.327386,1,1
2,0.06000,0.21000,0.11000,0.777070,0.420107,2,1
3,0.00000,0.15000,0.07000,0.693069,0.439358,3,1
4,0.03000,0.19000,0.06000,0.943782,0.676877,4,1
...,...,...,...,...,...,...,...
95,0.02500,0.45625,0.13125,0.482252,1.010616,95,3
96,0.03750,0.54375,0.23125,0.526316,0.844622,96,3
97,0.01250,0.55000,0.25625,-0.100629,0.780611,97,3
98,0.03125,0.45625,0.19375,-0.084507,0.754274,98,3


In [29]:
pg.anova(data=df_groups,dv='PR',between='group', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,0.036728,2,0.018364,121.853621,2.287958e-39,0.450719
1,Within,0.044760,297,0.000151,NaN,NaN,NaN


In [30]:
pg.pairwise_tukey(data=df_groups,dv='PR',between='group')

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,1,2,0.03520,0.043250,-0.008050,0.001736,-4.636771,0.000016,-0.578385
1,1,3,0.03520,0.016813,0.018388,0.001736,10.591134,0.000000,1.459456
2,2,3,0.04325,0.016813,0.026438,0.001736,15.227905,0.000000,2.603416


In [31]:
pg.anova(data=df_groups.loc[np.isfinite(df_groups.d)], dv='d',between='group', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,42.500055,2,21.250027,447.202742,2.567337e-90,0.750715
1,Within,14.112745,297,0.047518,NaN,NaN,NaN


In [32]:
pg.anova(data=df_groups,dv='Q',between='group', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,27.274891,2,13.637446,165.738459,4.553726e-49,0.527429
1,Within,24.438029,297,0.082283,NaN,NaN,NaN


In [33]:
pg.pairwise_tukey(data=df_groups,dv='Q',between='group')

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,1,2,0.687226,0.708729,-0.021504,0.040567,-0.530080,0.856636,-0.107523
1,1,3,0.687226,0.058621,0.628605,0.040567,15.495574,0.000000,1.868808
2,2,3,0.708729,0.058621,0.650108,0.040567,16.025654,0.000000,2.102677


### Symmetry Analysis

In [34]:
def test_sym(df_simu, testnum):
    df = df_simu.query(f"test == {testnum} and order >= 0").copy()
    df['correct'] = df['s_resp'] == df['correct_ans']
    df_order = df.groupby(["order","subject"]).correct.mean().to_frame(name="p_correct").reset_index()
    return df_order

In [35]:
df_simu_all = pd.concat([df_simu_g1, df_simu_g2, df_simu_g3])
df_order_recog = test_sym(df_simu_all, 1)
print(df_order_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_recog, dv='p_correct', subject='subject', within='order')

order
0    0.479417
1    0.475917
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,0.79182,99.0,two-sided,0.430359,0.15,0.116165


In [36]:
df_order_g1_recog = test_sym(df_simu_g1, 1)
print(df_order_g1_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g1_recog, dv='p_correct', subject='subject', within='order')

order
0    0.5841
1    0.5683
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,2.208649,99.0,two-sided,0.029506,1.125,0.364989


In [37]:
df_order_g1_cr = test_sym(df_simu_g1, 2)
print(df_order_g1_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g1_cr, dv='p_correct', subject='subject', within='order')

order
0    0.031637
1    0.038710
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-1.970658,99.0,two-sided,0.051556,0.707,-0.294586


In [38]:
df_order_g2_recog = test_sym(df_simu_g2, 1)
print(df_order_g2_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g2_recog, dv='p_correct', subject='subject', within='order')

order
0    0.251167
1    0.255333
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-0.608558,99.0,two-sided,0.544211,0.133,-0.071531


In [39]:
df_order_g2_cr = test_sym(df_simu_g2, 2)
print(df_order_g2_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g2_cr, dv='p_correct', subject='subject', within='order')

order
0    0.0425
1    0.0440
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-0.415635,99.0,two-sided,0.678577,0.12,-0.069471


In [40]:
df_order_g3_recog = test_sym(df_simu_g3, 1)
print(df_order_g3_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g3_recog, dv='p_correct', subject='subject', within='order')

order
0    0.519750
1    0.525875
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-0.82945,99.0,two-sided,0.408845,0.155,-0.104086


In [41]:
df_order_g3_cr = test_sym(df_simu_g3, 2)
print(df_order_g3_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g3_cr, dv='p_correct', subject='subject', within='order')

order
0    0.014750
1    0.018875
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-1.934427,99.0,two-sided,0.055915,0.662,-0.305189
